# MNW2 package example

In [1]:
import sys
import os
import numpy as np
try:
    import pandas as pd
except:
    pass
import flopy

### Make an MNW2 package from scratch

In [2]:
m = flopy.modflow.Modflow('mnw2example', model_ws='temp')
dis = flopy.modflow.ModflowDis(nrow=5, ncol=5, nlay=3, nper=3, top=10, botm=0, model=m)

### MNW2 information by node
(this could be prepared externally from well reconds and read in from a csv or excel file)
* this table has two multi-node wells, the first (well1) consisting of two nodes that are manually specified
(where the variable **rw** is specified by node)
* node that some variables that are constant for the whole well are also included (losstype, zpump, etc.)

In [3]:
node_data = pd.DataFrame([[1, 1, 9.5, 7.1, 'well1', 'skin', -1, 0, 0, 0, 1., 2., 5., 6.2],
                   [1, 1, 7.1, 5.1, 'well1', 'skin', -1, 0, 0, 0, 0.5, 2., 5., 6.2],
 [3, 3, 9.1, 3.7, 'well2', 'skin', -1, 0, 0, 0, 1., 2., 5., 4.1]], 
             columns=['i', 'j', 'ztop', 'zbotm', 'wellid', 'losstype', 'pumploc', 'qlimit', 'ppflag', 'pumpcap', 
                    'rw', 'rskin', 'kskin', 'zpump'])
node_data

,i,j,ztop,zbotm,wellid,losstype,pumploc,qlimit,ppflag,pumpcap,rw,rskin,kskin,zpump
0,1,1,9.5,7.1,well1,skin,-1,0,0,0,1.0,2,5,6.2
1,1,1,7.1,5.1,well1,skin,-1,0,0,0,0.5,2,5,6.2
2,3,3,9.1,3.7,well2,skin,-1,0,0,0,1.0,2,5,4.1


#### convert the DataFrame to a rec array for compatibility with flopy

In [4]:
node_data = node_data.to_records()
node_data

rec.array([(0, 1, 1, 9.5, 7.1, 'well1', 'skin', -1, 0, 0, 0, 1.0, 2.0, 5.0, 6.2),
 (1, 1, 1, 7.1, 5.1, 'well1', 'skin', -1, 0, 0, 0, 0.5, 2.0, 5.0, 6.2),
 (2, 3, 3, 9.1, 3.7, 'well2', 'skin', -1, 0, 0, 0, 1.0, 2.0, 5.0, 4.1)], 
          dtype=[('index', '<i8'), ('i', '<i8'), ('j', '<i8'), ('ztop', '<f8'), ('zbotm', '<f8'), ('wellid', 'O'), ('losstype', 'O'), ('pumploc', '<i8'), ('qlimit', '<i8'), ('ppflag', '<i8'), ('pumpcap', '<i8'), ('rw', '<f8'), ('rskin', '<f8'), ('kskin', '<f8'), ('zpump', '<f8')])

### Stress period information
(could also be developed externally)

In [5]:
stress_period_data = pd.DataFrame([[0, 'well1', 0],
                          [1, 'well1', 100.0],
                          [0, 'well2', 0],
                          [1, 'well2', 1000.]], columns=['per', 'wellid', 'qdes'])
stress_period_data

,per,wellid,qdes
0,0,well1,0
1,1,well1,100
2,0,well2,0
3,1,well2,1000


In [6]:
pers = stress_period_data.groupby('per')
stress_period_data = {i: pers.get_group(i).to_records() for i in [0, 1]}
stress_period_data

{0: rec.array([(0, 0, 'well1', 0.0), (2, 0, 'well2', 0.0)], 
           dtype=[('index', '<i8'), ('per', '<i8'), ('wellid', 'O'), ('qdes', '<f8')]),
 1: rec.array([(1, 1, 'well1', 100.0), (3, 1, 'well2', 1000.0)], 
           dtype=[('index', '<i8'), ('per', '<i8'), ('wellid', 'O'), ('qdes', '<f8')])}

### Make ``ModflowMnw2`` package object
* note that extraneous columns in node_data and stress_period_data are ignored
* if itmp is positive, it must equal the number of active wells being specified in ``stress_period_data``, otherwise the package class will raise an error.

In [7]:
mnw2 = flopy.modflow.ModflowMnw2(model=m, mnwmax=2,
                 node_data=node_data, 
                 stress_period_data=stress_period_data, 
                 itmp=[2, 2, -1], # reuse second per pumping for last stress period
                 )

In [8]:
# "nodtot" is computed automatically
mnw2.nodtot

3

In [9]:
pd.DataFrame(mnw2.node_data)

,k,i,j,ztop,zbotm,wellid,losstype,pumploc,qlimit,ppflag,...,hlim,qcut,qfrcmn,qfrcmx,hlift,liftq0,liftqmax,hwtol,liftn,qn
0,0,1,1,9.5,7.1,well1,skin,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,7.1,5.1,well1,skin,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,3,3,9.1,3.7,well2,skin,-1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
pd.DataFrame(mnw2.stress_period_data[0])

,wellid,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,well1,0,0,0,0,0,0,0
1,well2,0,0,0,0,0,0,0


In [11]:
pd.DataFrame(mnw2.stress_period_data[1])

,wellid,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,well1,100,0,0,0,0,0,0
1,well2,1000,0,0,0,0,0,0


In [12]:
tmp = flopy.modflow.ModflowMnw2(model=m,
                 itmp=[1, 1, -1], # reuse second per pumping for last stress period
                 )

### empty ``node_data`` and ``stress_period_data`` tables can also be generated by the package class, and then filled

In [13]:
node_data = tmp.get_empty_node_data(3)
node_data

rec.array([ (0, 0, 0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 (0, 0, 0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 (0, 0, 0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)], 
          dtype=[('k', '<i8'), ('i', '<i8'), ('j', '<i8'), ('ztop', '<f4'), ('zbotm', '<f4'), ('wellid', 'O'), ('losstype', 'O'), ('pumploc', '<i8'), ('qlimit', '<i8'), ('ppflag', '<i8'), ('pumpcap', '<i8'), ('rw', '<f4'), ('rskin', '<f4'), ('kskin', '<f4'), ('B', '<f4'), ('C', '<f8'), ('P', '<f4'), ('cwc', '<f4'), ('pp', '<f4'), ('pumplay', '<i8'), ('pumprow', '<i8'), ('pumpcol', '<i8'), ('zpump', '<f4'), ('hlim', '<f4'), ('qcut', '<i8'), ('qfrcmn', '<f4'), ('qfrcmx', '<f4'), ('hlift', '<f4'), ('liftq0', '<f4'), ('liftqmax', '<f4'), ('hwtol', '<f4'), ('l

### Mnw objects
at the base of the flopy mnw2 module is the **Mnw** object class, which describes a single multi-node well.
A list or dict of **Mnw** objects can be used to build a package (using the example above):
```
flopy.modflow.ModflowMnw2(model=m, mnwmax=2,
                 mnw=<dict or list of Mnw objects>,
                 itmp=[1, 1, -1], # reuse second per pumping for last stress period
                 )
```
or if node_data and stress_period_data are supplied, the **Mnw** objects are created on initialization of the ModflowMnw2 class instance, and assigned to the ```.mnw``` attribute, as items in a dictionary keyed by ```wellid```.

In [14]:
mnw2.mnw

{'well1': <flopy.modflow.mfmnw2.Mnw at 0x10ddf7978>,
 'well2': <flopy.modflow.mfmnw2.Mnw at 0x10ddf7a58>}

In [15]:
mnw2.mnw['well1'].__dict__

{'B': None,
 'C': 0,
 'P': 2.0,
 'aux': [],
 'cwc': None,
 'hlift': None,
 'hlim': None,
 'hwtol': None,
 'i': 1,
 'j': 1,
 'k': 0,
 'kskin': 5.0,
 'liftn': None,
 'liftq0': None,
 'liftqmax': None,
 'losstype': 'skin',
 'mnwpackage': 
     Multi-Node Well 2 Package Class
 
     Parameters
     ----------
     model : model object
         The model object (of type :class:'flopy.modflow.mf.Modflow') to which
         this package will be added.
     mnwmax : int
         The absolute value of MNWMAX is the maximum number of multi-node wells (MNW) to be simulated.
         If MNWMAX is a negative number, NODTOT is read.
     nodtot : int
         Maximum number of nodes.
         The code automatically estimates the maximum number of nodes (NODTOT)
         as required for allocation of arrays. However, if a large number of horizontal wells
         are being simulated, or possibly for other reasons, this default estimate proves to be inadequate,
         a new input option has been add

Note that Mnw object attributes for variables that vary by node are lists (e.g. ``rw`` above)

#### Each Mnw object has its own ``node_data`` and ``stress_period_data``

In [16]:
pd.DataFrame(mnw2.mnw['well1'].node_data)

,k,i,j,ztop,zbotm,wellid,losstype,pumploc,qlimit,ppflag,...,hlim,qcut,qfrcmn,qfrcmx,hlift,liftq0,liftqmax,hwtol,liftn,qn
0,0,1,1,9.5,7.1,well1,skin,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,7.1,5.1,well1,skin,-1,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Instead of a dict keyed by stress period, Mnw.stress_period_data is a recarray with pumping data listed by stress period for that well
* note that data for period 2, where ``itmp`` < 1, is shown (was copied from s.p. 1 during construction of the **Mnw** object)

In [17]:
pd.DataFrame(mnw2.mnw['well2'].stress_period_data)

,per,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,0,0,0,0,0,0,0,0
1,1,1000,0,0,0,0,0,0
2,1,1000,0,0,0,0,0,0


### Build the same package using only the ``Mnw`` objects

In [18]:
mnw2fromobj = flopy.modflow.ModflowMnw2(model=m, mnwmax=2,
                 mnw=mnw2.mnw,
                 itmp=[2, 2, -1], # reuse second per pumping for last stress period
                 )

In [19]:
pd.DataFrame(mnw2fromobj.node_data)

,k,i,j,ztop,zbotm,wellid,losstype,pumploc,qlimit,ppflag,...,hlim,qcut,qfrcmn,qfrcmx,hlift,liftq0,liftqmax,hwtol,liftn,qn
0,0,3,3,9.1,3.7,well2,skin,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,9.5,7.1,well1,skin,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,7.1,5.1,well1,skin,-1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
pd.DataFrame(mnw2fromobj.stress_period_data[0])

,wellid,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,well1,0,0,0,0,0,0,0
1,well2,0,0,0,0,0,0,0


In [21]:
pd.DataFrame(mnw2fromobj.stress_period_data[1])

,wellid,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,well1,100,0,0,0,0,0,0
1,well2,1000,0,0,0,0,0,0


### By default, the ``node_data`` and ``stress_period_data`` tables attached to the ``ModflowMnw2`` package class are definitive
* on writing of the package output (``mnw2.write_file()``), the **Mnw** objects are regenerated from the tables. This setting is controlled by the default argument ``use_tables=True``. To write the package file using the **Mnw** objects (ignoring the tables), use ``mnw2.write_file(use_tables=False)``. 

In [22]:
per1 = flopy.modflow.ModflowMnw2.get_empty_stress_period_data(itmp=2)
per1

rec.array([(0, 0.0, 0, 0.0, 0.0, 0, 0.0, 0.0), (0, 0.0, 0, 0.0, 0.0, 0, 0.0, 0.0)], 
          dtype=[('wellid', 'O'), ('qdes', '<f8'), ('capmult', '<i8'), ('cprime', '<f4'), ('hlim', '<f4'), ('qcut', '<i8'), ('qfrcmn', '<f8'), ('qfrcmx', '<f8')])

### Write an MNW2 package file and inspect the results

In [23]:
mnw2.write_file(os.path.join('temp/test.mnw2'))

In [24]:
junk = [print(l.strip('\n')) for l in open('temp/test.mnw2').readlines()]

# MNW2 for MODFLOW, generated by Flopy.
2 0 0
well1 -2
            skin -1 0 0 0
            -1.00 2.00 5.00
            9.50 7.10 2 2 1.00
            7.10 5.10 2 2 0.50
            6.20
well1 -2
            skin -1 0 0 0
            -1.00 2.00 5.00
            9.50 7.10 2 2 1.00
            7.10 5.10 2 2 0.50
            6.20
well2 -1
            skin -1 0 0 0
            1.00 2.00 5.00
            9.10 3.70 4 4
            4.10
2  Stress Period 1
well1 0.00
well2 0.00
2  Stress Period 2
well1 100.00
well2 1000.00
-1  Stress Period 3


### Load some example MNW2 packages

In [25]:
path = os.path.join('..', '..', 'examples', 'data', 'mf2005_test')
cpth = os.path.join('..', '..', 'autotest', 'temp')
m = flopy.modflow.Modflow('MNW2-Fig28', model_ws=cpth)
dis = flopy.modflow.ModflowDis.load(path + '/MNW2-Fig28.dis', m)

In [26]:
m.get_package_list()

['DIS']

In [27]:
mnw2pth = os.path.join(path, 'MNW2-Fig28.mnw2')
mnw2 = flopy.modflow.ModflowMnw2.load(mnw2pth, m)

In [28]:
pd.DataFrame(mnw2.node_data)

,k,i,j,ztop,zbotm,wellid,losstype,pumploc,qlimit,ppflag,...,hlim,qcut,qfrcmn,qfrcmx,hlift,liftq0,liftqmax,hwtol,liftn,qn
0,0,29,40,-5,-65,Well-A,SKIN,0,1,0,...,-7.5,-1,0.1,0.15,0,0,0,0,0,0


In [29]:
pd.DataFrame(mnw2.stress_period_data[0])

,wellid,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,Well-A,0,0,0,0,0,0,0


In [30]:
mnw2.mnw

{'Well-A': <flopy.modflow.mfmnw2.Mnw at 0x112fd4828>}

In [31]:
pd.DataFrame(mnw2.mnw['Well-A'].stress_period_data)

,per,qdes,capmult,cprime,hlim,qcut,qfrcmn,qfrcmx
0,0,0,0,0,0,0,0,0
1,1,-10000,0,0,0,0,0,0
2,2,-10000,0,0,0,0,0,0


In [32]:
path = os.path.join('..', '..', 'examples', 'data', 'mnw2_examples')
cpth = os.path.join('temp')
m = flopy.modflow.Modflow('br', model_ws=cpth)
mnw2 = flopy.modflow.ModflowMnw2.load(path + '/BadRiver_cal.mnw2', m)

In [33]:
df = pd.DataFrame(mnw2.node_data)
df.loc[:, df.sum(axis=0) != 0]

,i,j,ztop,zbotm,wellid,losstype,pumploc,rw,rskin,kskin,zpump
0,360,2,706.330017,699.330017,Hayward_bait_N,SKIN,-1,1,2,10,700.330017
1,360,6,705.289978,698.289978,Hayward_bait_NE,SKIN,-1,1,2,10,699.289978
2,208,148,520.119995,500.119995,NSP,SKIN,-1,1,2,10,501.119995
3,516,424,1079.910034,1072.910034,CFSP_ncp1,SKIN,-1,1,2,10,1073.910034
4,539,415,1093.010010,1003.010010,CFSP_of,SKIN,-1,1,2,10,1004.010010
5,515,424,1077.900024,1074.900024,CFSP_ncp2,SKIN,-1,1,2,10,1075.900024
6,294,503,181.630005,161.630005,BR_Birch1,SKIN,-1,1,2,10,162.630005
7,295,503,179.119995,159.119995,BR_Birch2,SKIN,-1,1,2,10,160.119995
8,175,342,400.220001,312.220001,BR_Diaperville1,SKIN,-1,1,2,10,313.220001
9,174,342,399.119995,312.119995,BR_DIaperville2,SKIN,-1,1,2,10,313.119995
